# Import Libraries

In [85]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer, required
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import copy
import matplotlib.gridspec as gridspec    
import os
from tqdm import tqdm
from itertools import chain
import matplotlib.pyplot as plt

## Choose the batch size

In [97]:
nEpoch = 12
batch_size = 1 #replace it with 8 if you want to compare optimizers for minibatch size 8

## Dataset creation

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(image_dim))])
train_set = torchvision.datasets.MNIST(root='./data/MNIST', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)

# Define Architecture

In [5]:
class Encoder(nn.Module):

    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 500)  #layer with 1000 nodes is removed
        self.fc3 = nn.Linear(500, 250)
        self.fc4 = nn.Linear(250, 30)
             
    def forward(self, x):
        out = self.fc1(x)
        out = torch.sigmoid(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)
        out = self.fc4(out)
        return out

class Decoder(nn.Module):

    def __init__(self, input_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 250)
        self.fc2 = nn.Linear(250, 500)   #layer with 1000 nodes is removed
        self.fc4 = nn.Linear(500, 784)

    def forward(self, x):
        out = self.fc1(x)
        out = torch.sigmoid(out)
        out = self.fc2(out)
        out = torch.sigmoid(out)
        out = self.fc4(out)
        out = torch.sigmoid(out)
        return out
    
    
device = torch.device("cuda:0")
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

enc_dim = 30                      #given in paper
image_dim = 784                   # 28*28 size of input image flattened to 784       
enc = Encoder(image_dim).to(device)
dec = Decoder(enc_dim).to(device)

In [6]:
class AccSGD(Optimizer):
    r"""Implements the algorithm proposed in https://arxiv.org/pdf/1704.08227.pdf, which is a provably accelerated method 
    for stochastic optimization. This has been employed in https://openreview.net/forum?id=rJTutzbA- for training several 
    deep learning models of practical interest. This code has been implemented by building on the construction of the SGD 
    optimization module found in pytorch codebase.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate (required)
        kappa (float, optional): ratio of long to short step (default: 1000)
        xi (float, optional): statistical advantage parameter (default: 10)
        smallConst (float, optional): any value <=1 (default: 0.7)
    Example:
        >>> from AccSGD import *
        >>> optimizer = AccSGD(model.parameters(), lr=0.1, kappa = 1000.0, xi = 10.0)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

    def __init__(self, params, lr=required, kappa = 1000.0, xi = 10.0, smallConst = 0.7, weight_decay=0):
        defaults = dict(lr=lr, kappa=kappa, xi=xi, smallConst=smallConst,
                        weight_decay=weight_decay)
        super(AccSGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AccSGD, self).__setstate__(state)

    def step(self, closure=None):
        """ Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            large_lr = (group['lr']*group['kappa'])/(group['smallConst'])
            Alpha = 1.0 - ((group['smallConst']*group['smallConst']*group['xi'])/group['kappa'])
            Beta = 1.0 - Alpha
            zeta = group['smallConst']/(group['smallConst']+Beta)
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                param_state = self.state[p]
                if 'momentum_buffer' not in param_state:
                    param_state['momentum_buffer'] = copy.deepcopy(p.data)
                buf = param_state['momentum_buffer']
                buf.mul_((1.0/Beta)-1.0)
                buf.add_(-large_lr,d_p)
                buf.add_(p.data)
                buf.mul_(Beta)

                p.data.add_(-group['lr'],d_p)
                p.data.mul_(zeta)
                p.data.add_(1.0-zeta,buf)

        return loss

In [7]:
def lossfn(optimizer):    
    loss_all2 = []
    for epoch in range(nEpoch):
        losses = []
        trainloader = tqdm(train_loader)
        
        for i, data in enumerate(trainloader, 0):
            if i * batch_size >= 10000:
                break
            else:
                inputs, _ = data
                optimizer.zero_grad()

                inputs = inputs.to(device)
                z = enc(inputs)
                outputs = dec(z)

                loss = F.mse_loss(outputs, inputs, size_average=False) / inputs.shape[0]
                loss.backward()
                optimizer.step()

                # keep track of the loss and update the stats
                losses.append(loss.item())
                trainloader.set_postfix(loss=np.mean(losses), epoch=epoch)
        loss_all2.append(np.sqrt(np.mean(losses)))
    return loss_all2

# Various Gradient descent functions

In [8]:
def SGD(lr):
    optimizer = optim.SGD(chain(enc.parameters(), dec.parameters()), lr)
    return lossfn(optimizer)

In [84]:
def ADAM( lr):
    optimizer = optim.Adam(chain(enc.parameters(), dec.parameters()), lr)
    return lossfn(optimizer)
    

In [44]:
def ASGD(lr, kappa, xi):
    optimizer = AccSGD(chain(enc.parameters(), dec.parameters()), lr, kappa, xi)
    return lossfn(optimizer)

In [45]:
def NAG( lr, momentum):
    optimizer = optim.SGD(chain(enc.parameters(), dec.parameters()), lr, momentum, nesterov = True)
    return lossfn(optimizer)

In [46]:
def HB(lr, momentum):
    optimizer = optim.SGD(chain(enc.parameters(), dec.parameters()), lr, momentum)
    return lossfn(optimizer)

# Plots for gridsearch

### Plotting function

In [82]:
def plot(loss, name, length):
    loss_init = max(loss[0,:])
    dif = np.zeros(np.size(loss[0, :]))
    dif = loss_init * np.ones(np.size(loss[0, :])) - loss[0, :]
    for i in range(np.size(loss[:, 0])):
        for j in range(np.size(loss[0, :])):
            loss[i, j] = loss[i, j] + dif[j]
    loss[0, :] = loss_init * np.ones(np.size(loss[0, :]))
    
    plt.figure()
    plt.title('Choosing learning rate of '+name)
    plt.xlabel('Epochs')
    plt.ylabel('MSE lose')
    x = np.linspace(0, nEpoch, nEpoch)
    for i in range(length):
        plt.plot(x, loss[:, i], label = str(i))
        plt.legend()
    plt.savefig(name+'.png')
    plt.show

# SGD

In [ ]:
lr_grid = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]   

i = 0
loss = np.zeros((nEpoch, 10))
for lr in lr_grid:
    loss_SGD = SGD( lr)          
    loss[:, i] = loss_SGD
    i += 1
plot(loss, 'SGD', int(np.size(lr_grid)))

# Adam

In [93]:
lr_grid = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01]

i = 0
loss = np.zeros((nEpoch, int(np.size(lr_grid))))
for lr in lr_grid:
    loss_ADAM = ADAM(lr)          
    loss[:, i] = loss_ADAM
    i += 1
    
plot(loss, 'Adam', int(np.size(lr_grid)*np.size(momentum_grid)))

# NAG

In [94]:
lr_grid = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
momentum_grid = [0, 0.5, 0.75, 0.9, 0.95, 0.9]

i = 0
loss = np.zeros((nEpoch,int(np.size(lr_grid)*np.size(momentum_grid))) )
for lr in lr_grid:
    for momentum in momentum_grid:
        loss_NAG = NAG(lr, momentum)          
        loss[:, i] = loss_NAG
        i += 1
        
plot(loss, 'NAG', int(np.size(lr_grid)*np.size(momentum_grid)))

# HB

In [95]:
lr_grid = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
momentum_grid = [0, 0.5, 0.75, 0.9, 0.95, 0.9]

i = 0
loss = np.zeros((nEpoch, int(np.size(lr_grid)*np.size(momentum_grid))))
for lr in lr_grid:
    for momentum in momentum_grid:
        loss_HB = HB(lr, momentum)          
        loss[:, i] = loss_HB
        i += 1
plot(loss, 'HB', int(np.size(lr_grid)*np.size(momentum_grid)))

# ASGD

In [96]:
lr_grid = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
ls_grid = [100, 1000]
ap_grid = [ 2.5, 5.0, 10.0, 20]

i = 0
loss = np.zeros((nEpoch, int(np.size(lr_grid)*np.size(ls_grid)*np.size(ap_grid))))
for lr in lr_grid:
    for ls in ls_grid:
        for ap in ap_grid:
            loss_ASGD = ASGD(lr, ls, ap)
            loss[:, i] = loss_ASGD
            i += 1
            
plot(loss, 'ASGD', int(np.size(lr_grid)*np.size(momentum_grid)))

Note: rerun the grid search box many times to obatain again best parameters where the algorithms perform optimally

If you rerun then do the following:
First run the below cell.
After that obtain two adjecent values where the algo perform best.
Then devide those parameters into minor subsets to obtain better graphs.